### Setup Configurations

In [1]:
import import_ipynb
import setup

importing Jupyter notebook from setup.ipynb


### Get form inputs for form of Type Task 1

(a)  a stock's ticker symbol  [OR PART OF THE COMPANY'S / ENTITY'S NAME] <br>
(b)  the form type needed to conduct the search  [OPTIONAL TO THE USER; THIS SEARCH WILL BE LIMITED TO 10-Ks AND 10-Qs AND THEIR VARIATIONS, e.g. 10-KSB, 10-K405, 10-QSB, etc.], <br>
(c)  the [FINANCIAL STATEMENT REPORT OR DISCLOSURE] information sought within the form <br>
(d)  optionally, the filing period <br>


In [2]:
# Get form inputs
form_inputs = setup.get_inputs_tsk1()

Enter Stock Ticker Symbol>>  aapl
Enter Filing Type Required (10-K,10-Q,etc)>>  10-K
Do you want to search for a Financial Table (T) or Particular Item in Document (I),Please Enter (T) or (I)>>  t
Enter Document Required>> apple
To enter Date of Filing only enter one date. To enter Date period enter 2 dates.  
Filed FROM Date: (YYYY-MM-DD)>>  
Filed TO Date: (YYYY-MM-DD)>>  


In [3]:
#generate dynamic search url for queries
search_url = setup.generate_search_url(form_inputs)

SEARCH_URL https://www.sec.gov/edgar/search/#/q=%2522apple%2522&category=custom&entityName=aapl&forms=10-K&startdt=2019-03-02&enddt=2022-03-01


In [6]:
#fetch all search results for query
print(setup.get_search_results(search_url))

<table class="table" style="display: none;">
<thead style="display: none;">
<tr style="display: none;">
<th class="filetype" id="filetype" style="display: none;">Form &amp; File</th>
<th class="filed" style="display: none;">Filed</th>
<th class="enddate" style="display: none;">Reporting for</th>
<th class="entity-name" style="display: none;">Filing entity/person</th>
<th class="cik" style="display: none;">CIK</th>
<th class="located" style="display: none;">Located</th>
<th class="incorporated" style="display: none;">Incorporated</th>
<th class="file-num" style="display: none;">File number</th>
<th class="film-num" style="display: none;">Film number</th>
</tr>
</thead>
<tbody style="display: none;">
</tbody>
</table>
None
